In [ ]:
# Initialize trace environment and set up some helper methods
from helpers import trace_env
from helpers import image_formats
from helpers import jupyter_helpers
from helpers import replay
from helpers import trace as trace_info
from helpers import vulkan
from helpers import json_html
import matplotlib
import os
import ipywidgets
import numpy as np

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [16, 9]

pathdir = globals()['_dh'][0]
env = trace_env.trace_env(os.path.normpath(os.path.join(pathdir, "..", "build", "RelWithDebInfo")))
env.ignore_debug_messages = True
env.ignore_layer_info_messages = True

In [ ]:
# Select and load trace file
from ipyfilechooser import FileChooser

fc = FileChooser()
if env.current_trace:
    fc.default_path = os.path.dirname(env.current_trace)
    fc.default_filename = os.path.basename(env.current_trace)

fc.filter_pattern = "*.trace"
fc.title = "<b>Select Trace</b>"
trace = None
def change_file(chooser):
    global trace
    current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
    trace =  trace_info.load_trace(env, current_trace)
fc.register_callback(change_file)
display(fc)

In [ ]:
import ipywidgets
b = ipywidgets.widgets.HTML("")
def show(i, to_show):
    b.value = json_html.get_html_for_commands(trace.real_commands[i:i+to_show])

ipywidgets.interact(show, to_show=ipywidgets.fixed(20), i=ipywidgets.widgets.IntSlider(min=0, max=len(trace.real_commands) - 10), value=0)
display(b)

In [ ]:

opts = replay.replay_options(env, trace)
times = []
def on_data(timestamp, data):
    times.append(data['time'])
def on_message(timestamp, level, message):
    pass
opts.add_layer(os.path.abspath("pipeline_time.cpp"), {}, data_callback=on_data)
opts.set_message_callback(on_message)

replay.replay(env, opts)

fig, ax =  matplotlib.pyplot.subplots()
counts, bins, patches = ax.hist(times)
ax.set_xticks(bins)
for count, rect, bin in zip(counts, patches, bins):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height+0.01, str(int(count)), ha='center', va='bottom')
matplotlib.pyplot.show()

In [ ]:
sorted_times = sorted([x for x in range(len(times))], key=lambda x: times[x])
all_pipeline_creations = [x for x in trace.commands if x['name'] == 'vkCreateGraphicsPipelines']

ipywidgets.widgets.HTML(json_html.get_html_for_commands(list(np.array(all_pipeline_creations)[sorted_times[:10]])))

In [ ]:
import bisect
from collections import defaultdict

submissions = trace.get_trace_from_submission_pov()
all_submitted_commands = [y.commands for x in submissions[0].queue_submits for y in x.submissions]
all_draws = [[y for y in x if "Draw" in trace.commands[y[1]]['name']] for x in all_submitted_commands]
all_pipeline_binds = [[y for y in x if "vkCmdBindPipeline" == trace.commands[y[1]]['name'] and trace.commands[y[1]]['pipelineBindPoint'] == 0] for x in all_submitted_commands]

all_used_pipelines = []
for i in range(len(all_draws)):
    all_used_pipelines.append([all_pipeline_binds[i][bisect.bisect_left(all_pipeline_binds[i], x)-1] for x in all_draws[i]])

pipelines = defaultdict(int)
for x in all_used_pipelines:
    for y in x:
        pipelines[trace.commands[y[1]]['pipeline']] += 1

fig, ax =  matplotlib.pyplot.subplots()
ax.bar([str(x) for x in pipelines.keys()], [x for x in pipelines.values()])
matplotlib.pyplot.show()

In [ ]:
top_used = sorted([(x, v) for x, v in pipelines.items()], key=lambda x: x[1], reverse=True)
top_used_pipelines = [next(x for x in all_pipeline_creations if x['pPipelines'][0] == y[0]) for y in top_used[:10]]

ipywidgets.widgets.HTML(json_html.get_html_for_commands(top_used_pipelines))